# Imports

In [1]:
import math
import pandas as pd
import pennylane as qml
import time

from keras.datasets import mnist
from matplotlib import pyplot as plt
from pennylane import numpy as np
from pennylane.templates import AmplitudeEmbedding, AngleEmbedding
from pennylane.templates.subroutines import ArbitraryUnitary
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Model Params

In [2]:
np.random.seed(131)
initial_params = np.random.random([105])

INITIALIZATION_METHOD = 'Angle'
BATCH_SIZE = 20
EPOCHS = 400

STEP_SIZE = 0.01
BETA_1 = 0.9
BETA_2 = 0.99
EPSILON = 0.00000001

TRAINING_SIZE = 0.78
VALIDATION_SIZE = 0.07
TEST_SIZE = 1-TRAINING_SIZE-VALIDATION_SIZE

initial_time = time.time()

# Import dataset

In [3]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()
examples = np.append(train_X, test_X, axis=0)
examples = examples.reshape(70000, 28*28)
classes = np.append(train_y, test_y)

In [4]:
x = []
y = []
for (example, label) in zip(examples, classes):
    if label == 2:
        x.append(example)
        y.append(-1)
    elif label == 7:
        x.append(example)
        y.append(1)

In [5]:
x = np.array(x)
y = np.array(y)

# Normalize pixels values
x = x / 255

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=TEST_SIZE, shuffle=True)

In [6]:
validation_indexes = np.random.random_integers(len(X_train), size=(math.floor(len(X_train)*VALIDATION_SIZE),))
X_validation = [X_train[n] for n in validation_indexes]
y_validation = [y_train[n] for n in validation_indexes]

In [7]:
pca = PCA(n_components=8)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_validation = pca.transform(X_validation)
X_test = pca.transform(X_test)

preprocessing_time = time.time()

# Circuit creation

In [8]:
device = qml.device("default.qubit", wires=8)

In [9]:
@qml.qnode(device)
def circuit(features, params):
    # Load state
    if INITIALIZATION_METHOD == 'Amplitude':
        AmplitudeEmbedding(features=features, wires=range(8), normalize=True, pad_with=0.)
    else:
        AngleEmbedding(features=features, wires=range(8), rotation='Y')

    # First layer
    ArbitraryUnitary(params[:15], wires=[0, 1])
    ArbitraryUnitary(params[15:30], wires=[2, 3])
    ArbitraryUnitary(params[30:45], wires=[4, 5])
    ArbitraryUnitary(params[45:60], wires=[6, 7])

    # Second layer
    ArbitraryUnitary(params[60:75], wires=[1, 2])
    ArbitraryUnitary(params[75:90], wires=[5, 6])

    # Third layer
    ArbitraryUnitary(params[90:105], wires=[2, 5])

    # Measurement
    return qml.expval(qml.PauliZ(5))

## Circuit example

In [10]:
features = X_train[0]
print(f"Inital parameters: {initial_params}\n")
print(f"Example features: {features}\n")
print(f"Expectation value: {circuit(features, initial_params)}\n")
print(circuit.draw())

Inital parameters: [0.65015361 0.94810917 0.38802889 0.64129616 0.69051205 0.12660931
 0.23946678 0.25415707 0.42644165 0.83900255 0.74503365 0.38067928
 0.26169292 0.05333379 0.43689638 0.20897912 0.59441102 0.09890353
 0.22409353 0.5842624  0.95908107 0.20988382 0.66133746 0.50261295
 0.32029143 0.12506485 0.80688893 0.98696002 0.54304141 0.23132314
 0.60351254 0.17669598 0.88653747 0.58902228 0.72117264 0.27567029
 0.78811469 0.1326223  0.39971595 0.62982409 0.42404345 0.16187284
 0.52034418 0.6070413  0.5808057  0.82111597 0.98499188 0.93449492
 0.90305486 0.3380262  0.78324429 0.74373474 0.58058546 0.43266356
 0.66792795 0.23668741 0.45173663 0.91999741 0.96687301 0.76905057
 0.32671177 0.62283984 0.19160224 0.24832171 0.11683869 0.01032549
 0.84869439 0.53066622 0.49383494 0.65230764 0.22667618 0.61716246
 0.2005521  0.49093199 0.94852991 0.98197027 0.17315162 0.74251972
 0.38646803 0.65110712 0.7434636  0.24777039 0.49229597 0.42474983
 0.29006936 0.98668888 0.33945016 0.1997036

# Accuracy test definition

In [11]:
def measure_accuracy(x, y, circuit_params):
    class_errors = 0

    for example, example_class in zip(x, y):
        predicted_value = circuit(example, circuit_params)

        if (example_class > 0 and predicted_value <= 0) or (example_class <= 0 and predicted_value > 0):
            class_errors += 1

    return 1 - (class_errors/len(y))

# Training

In [12]:
params = initial_params
opt = qml.AdamOptimizer(stepsize=STEP_SIZE, beta1=BETA_1, beta2=BETA_2, eps=EPSILON)
test_accuracies = []
best_validation_accuracy = 0.0
best_params = []

for i in range(len(X_train)):
    features = X_train[i]
    expected_value = y_train[i]

    def cost(circuit_params):
        value = circuit(features, circuit_params)
        return ((expected_value - value) ** 2)/len(X_train)

    params = opt.step(cost, params)

    if i % BATCH_SIZE == 0:
        print(f"epoch {i//BATCH_SIZE}")

    if i % (10*BATCH_SIZE) == 0:
        current_accuracy = measure_accuracy(X_validation, y_validation, params)
        test_accuracies.append(current_accuracy)
        print(f"accuracy: {current_accuracy}")

        if current_accuracy > best_validation_accuracy:
            print("best accuracy so far!")
            best_validation_accuracy = current_accuracy
            best_params = params

    if len(test_accuracies) == 30:
        print(f"test_accuracies: {test_accuracies}")

        if np.allclose(best_validation_accuracy, test_accuracies[0]):
            params = best_params
            break

        del test_accuracies[0]

epoch 0
accuracy: 0.4028268551236749
best accuracy so far!
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
accuracy: 0.6501766784452296
best accuracy so far!
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19
epoch 20
accuracy: 0.7255594817432274
best accuracy so far!
epoch 21
epoch 22
epoch 23
epoch 24
epoch 25
epoch 26
epoch 27
epoch 28
epoch 29
epoch 30
accuracy: 0.71849234393404
epoch 31
epoch 32
epoch 33
epoch 34
epoch 35
epoch 36
epoch 37
epoch 38
epoch 39
epoch 40
accuracy: 0.7396937573616018
best accuracy so far!
epoch 41
epoch 42
epoch 43
epoch 44
epoch 45
epoch 46
epoch 47
epoch 48
epoch 49
epoch 50
accuracy: 0.7608951707891637
best accuracy so far!
epoch 51
epoch 52
epoch 53
epoch 54
epoch 55
epoch 56
epoch 57
epoch 58
epoch 59
epoch 60
accuracy: 0.7726737338044758
best accuracy so far!
epoch 61
epoch 62
epoch 63
epoch 64
epoch 65
epoch 66
epoch 67
epoch 68
epoch 69
epoch 70
accuracy: 0.767962308598351
epoch 71
e

epoch 341
epoch 342
epoch 343
epoch 344
epoch 345
epoch 346
epoch 347
epoch 348
epoch 349
epoch 350
accuracy: 0.7455830388692579
test_accuracies: [0.7726737338044758, 0.767962308598351, 0.751472320376914, 0.7868080094228505, 0.7420494699646643, 0.773851590106007, 0.7396937573616018, 0.7491166077738516, 0.7656065959952886, 0.7891637220259129, 0.7597173144876326, 0.7585394581861014, 0.7974087161366313, 0.7773851590106007, 0.7561837455830389, 0.8150765606595995, 0.7809187279151943, 0.751472320376914, 0.7714958775029446, 0.7361601884570083, 0.7856301531213192, 0.7597173144876326, 0.7773851590106007, 0.7809187279151943, 0.7114252061248527, 0.773851590106007, 0.7656065959952886, 0.767962308598351, 0.7762073027090695, 0.7455830388692579]
epoch 351
epoch 352
epoch 353
epoch 354
epoch 355
epoch 356
epoch 357
epoch 358
epoch 359
epoch 360
accuracy: 0.7714958775029446
test_accuracies: [0.767962308598351, 0.751472320376914, 0.7868080094228505, 0.7420494699646643, 0.773851590106007, 0.7396937573616

epoch 455
epoch 456
epoch 457
epoch 458
epoch 459
epoch 460
accuracy: 0.7820965842167256
test_accuracies: [0.7585394581861014, 0.7974087161366313, 0.7773851590106007, 0.7561837455830389, 0.8150765606595995, 0.7809187279151943, 0.751472320376914, 0.7714958775029446, 0.7361601884570083, 0.7856301531213192, 0.7597173144876326, 0.7773851590106007, 0.7809187279151943, 0.7114252061248527, 0.773851590106007, 0.7656065959952886, 0.767962308598351, 0.7762073027090695, 0.7455830388692579, 0.7714958775029446, 0.7703180212014135, 0.7820965842167256, 0.773851590106007, 0.7773851590106007, 0.7550058892815077, 0.7550058892815077, 0.784452296819788, 0.7750294464075382, 0.7667844522968198, 0.7820965842167256]
epoch 461
epoch 462
epoch 463
epoch 464
epoch 465
epoch 466
epoch 467
epoch 468
epoch 469
epoch 470
accuracy: 0.7691401648998822
test_accuracies: [0.7974087161366313, 0.7773851590106007, 0.7561837455830389, 0.8150765606595995, 0.7809187279151943, 0.751472320376914, 0.7714958775029446, 0.7361601884

In [13]:
print("Optimized rotation angles: {}".format(params))

training_time = time.time()

Optimized rotation angles: [ 0.54968543  1.81789521  0.38385958  0.93530088  1.04404724 -0.35311928
 -0.2242418  -0.24721168  0.15538279  1.24081545  0.8464857   0.29238167
  0.51402311  0.55134583  0.08842742  1.25085023  1.52482392 -0.64801379
 -0.01494413  1.55404333  0.88002997  1.16367311  0.48532641  0.15197853
  1.36048655  0.12049302  0.60731044  0.94591864  0.22481264  0.23132314
  0.37952591  0.16288073  1.9835397   0.78966306  0.82855664 -0.14463466
  1.218971   -0.2144155  -0.35088115  0.95661621  0.8654488  -0.42134539
 -0.19099776  0.62233572  0.18194023  1.06452421  0.29384573  1.16168306
  0.74994316  0.48876791  1.34647782  1.0123925  -0.05178364 -0.09292688
  0.87903183 -0.26515112  0.63078798  0.11291333  0.65582216  0.76905057
  0.47342434  0.78532657 -0.17077623  0.90775198  0.50364106 -0.29332731
  0.75306058  0.59690518  0.22333334  1.25470117  0.19774379  0.55273101
  0.1411071   0.78623147  0.9989018   0.97357261  0.75576562  1.0978212
  0.64540729  1.51607043 

# Testing

In [14]:
accuracy = measure_accuracy(X_test, y_test, params)
print(accuracy)

test_time = time.time()

0.8086794213719085


In [15]:
print(f"pre-processing time: {preprocessing_time-initial_time}")
print(f"training time: {training_time - preprocessing_time}")
print(f"test time: {test_time - training_time}")
print(f"total time: {test_time - initial_time}")

pre-processing time: 2.8212294578552246
training time: 4141.519684314728
test time: 164.56386280059814
total time: 4308.904776573181
